In [16]:
import pdfplumber
import pandas as pd

filename = "DTU_vaegt_tbl1.pdf"

### Single page approach

In [30]:
with pdfplumber.open(filename) as pdf:
    print(pdf.pages)
    page = pdf.pages[0]

    cells = page.find_tables()[0].cells  # Draw rectangles around the cells
    print(cells)

    page_tables = page.extract_tables({
        "vertical_strategy": "explicit",
        "horizontal_strategy": "explicit",
        "explicit_vertical_lines": cells,
        "explicit_horizontal_lines": cells,
    })

# page_tables = page_tables[0] #Removing outermost nesting level
# page_tables = page_tables[1:] #Removing first item since it is irrelevant
# # page_tables = [[replace_newlines(cell) for cell in row] for row in page_tables] #Replace newline characters with whitespace
headers = page_tables[0] #Extracting headers
page_tables = page_tables[1:] #Removing headers from data
df = pd.DataFrame(page_tables, columns=headers)
# df = df.applymap(replace_newlines) 
print(df.to_markdown())

[<Page:1>]
[(61.380424559848485, 143.72950894999997, 352.0738963888889, 159.72812), (61.380424559848485, 159.72812, 352.0738963888889, 177.4053364679245), (61.380424559848485, 177.4053364679245, 66.3261404864865, 191.1539064827586), (61.380424559848485, 191.1539064827586, 352.0738963888889, 208.98991024285715), (61.380424559848485, 208.98991024285715, 66.3261404864865, 221.53571611428572), (61.380424559848485, 221.53571611428572, 352.0738963888889, 239.5353676935484), (61.380424559848485, 239.5353676935484, 66.3261404864865, 252.4250883125), (61.380424559848485, 252.4250883125, 352.0738963888889, 270.03008305555556), (61.380424559848485, 270.03008305555556, 66.3261404864865, 283.2319411206896), (61.380424559848485, 283.2319411206896, 352.0738963888889, 300.7611830555556), (61.380424559848485, 300.7611830555556, 66.3261404864865, 313.96307560344826), (61.380424559848485, 313.96307560344826, 352.0738963888889, 336.85925342592594), (61.380424559848485, 336.85925342592594, 66.3261404864865

TypeError: can only concatenate tuple (not "int") to tuple

In [ ]:
#Drawing 


cells = []

with pdfplumber.open(filename) as pdf:
    print(pdf.pages)
    page = pdf.pages[0]
    element_type = page.rects
    #TODO: 
    # 1. Ignore rects with height = 0? (or under certain threshold)
    # 2. Ignore nested rects
    im = page.to_image(resolution = 400)



    for i in range(0,len(element_type)):
    # for i in range(0,20):
    #Selected rects
        rect = element_type[i]
        # if rect['height'] < 1 or rect['width'] < 1: continue #1 Ignore small rectangles
        cells.append(rect)
        # print(f"Rect index: {i}")
        # print(f"Rect height: {rect['height']}")
        # print(f"Rect width: {rect['width']}")
    im = im.draw_rects(cells)
    im.show()

[<Page:1>]
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>

In [ ]:
import pdfplumber

rects = []

def create_rect(t: tuple) -> dict:
    (x0,y0,x1,y1) = t
    return {
        'x0': x0,
        'x1': x1,
        'y0': y0,
        'y1': y1,
        'object_type': 'rect'
    }



# Open the PDF file
with pdfplumber.open('DTU_vaegt_tbl1.pdf') as pdf:
    # Get the first page
    page = pdf.pages[0]
    for i in range(0,len(page.rects)):
        print(page.rects[i])

        page.rects

    # Create an image from the page
    im = page.to_image(resolution=150)
    
    my_cells = page.find_tables()[0].cells

    for cell in my_cells:
        rect = create_rect(cell)
        rects.append(rect)
    
    im.draw_rects(rects)  # Draw rectangles around the cells
    im.show()


{'x0': 60.68635, 'y0': 697.8101, 'x1': 61.607132099999994, 'y1': 698.7308821, 'width': 0.9207820999999967, 'height': 0.9207820999999967, 'pts': [(60.68635, 144.18989999999997), (61.607132099999994, 144.18989999999997), (61.607132099999994, 143.26911789999997), (60.68635, 143.26911789999997)], 'linewidth': 0, 'stroke': False, 'fill': True, 'evenodd': False, 'stroking_color': 0, 'non_stroking_color': (0.969, 0.588, 0.275), 'mcid': None, 'tag': None, 'object_type': 'rect', 'page_number': 1, 'path': [('m', (60.68635, 144.18989999999997)), ('l', (61.607132099999994, 144.18989999999997)), ('l', (61.607132099999994, 143.26911789999997)), ('l', (60.68635, 143.26911789999997)), ('h',)], 'dash': None, 'top': 143.26911789999997, 'bottom': 144.18989999999997, 'doctop': 143.26911789999997}
{'x0': 60.68635, 'y0': 697.8101, 'x1': 61.607132099999994, 'y1': 698.7308821, 'width': 0.9207820999999967, 'height': 0.9207820999999967, 'pts': [(60.68635, 144.18989999999997), (61.607132099999994, 144.1898999999

KeyError: 'top'